In [9]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

from const import DATA_DIR

stockpiles_path = DATA_DIR / 'nuclear-warhead-stockpiles-lines/nuclear-warhead-stockpiles-lines.csv'
df_stockpiles = pd.read_csv(stockpiles_path)

pio.templates.default = "plotly_dark"

major_powers_colormap = {
    'United States': '#56CCF2',
    'Russia':        '#FF6B6B',
    'China':         '#FFE66D',
    'France':        '#BB6BD9',
    'United Kingdom':'#95A5A6',
    'World':         '#F0F3F4' 
}
major_powers = list(major_powers_colormap.keys())

stockpiles_filtered = df_stockpiles.copy()

# Group non-major powers as "Other"
stockpiles_filtered = stockpiles_filtered[stockpiles_filtered['Entity'] != 'World']
stockpiles_filtered.loc[~stockpiles_filtered['Entity'].isin(major_powers), 'Entity'] = 'Other'
stockpiles_filtered = stockpiles_filtered.groupby(['Year', 'Entity'], as_index=False)['Number of nuclear warheads'].sum()

# Sorting
max_warheads = stockpiles_filtered.groupby('Entity')['Number of nuclear warheads'].max()
sorted_entities = max_warheads.sort_values(ascending=False).index.tolist()

# plot
fig1 = px.line(stockpiles_filtered, 
            x='Year', 
            y='Number of nuclear warheads', 
            width=800,
            height=400,
            color='Entity',
            title='<b>The Rise & Fall of Nuclear Arsenals</b><br><sub>Tracking the accumulation of warheads across the Cold War and beyond</sub>',
            template='plotly_dark',
            category_orders={'Entity': sorted_entities},
            color_discrete_map={
                    'United States': '#95E1D3', 'Russia': '#FF6B6B', 'China': '#FFE66D',
                    'United Kingdom': "#FFFFFF", 'France': '#C7CEEA'
                })

fig1.update_layout(hovermode="x unified", xaxis_title=None)
fig1.update_traces(hovertemplate='%{y}')
fig1.show()

In [ ]:
from const import VISUALIZATIONS_DIR

fig1.write_html(VISUALIZATIONS_DIR / "rise-and-fall-of-nuclear-arsenals.html")